In [1]:
import geopandas as gpd
import folium
import json
from tqdm import tqdm
import re
import plotly.express as px

from custom_functions.gtfs_methods import detect_direction, match_schedule_for_service_line
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

## Exporting sample

In [2]:
sp = pd.read_pickle("data/computed/finally_matches_subsampled.pkl")
sp.shape

(2121, 81)

In [3]:
sp.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft,label,date_normalized,stop_id_clean_c,descr_fr,descr_nl,geometry,geom_position_before,dist_m,date,time,distance_from_point,stop_id__current,stop_id_cleaned,stop_id__terminus,stop_code__terminus,stop_name__terminus,stop_desc__terminus,stop_lat__terminus,stop_lon__terminus,zone_id__terminus,stop_url__terminus,location_type__terminus,parent_station__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,lag_-1,lag_+1,t_lag_before,t_diff(sec),adjusted_arrival_time(ts),adjusted_arrival_time,diff_time,vehicule_id,approx_rec
0,112947202236269500,07:30:56,07:30:56,8733,1,0,0,NaN,GARE DE L'OUEST,NaN,50.848999,4.320948,NaN,NaN,0.0,2.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,26738,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8733,GARE DE L'OUEST,WESTSTATION,POINT (146633.5 170956.4),None,NaN,11-09-2021,07:25:38,0.0,8733,8733,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,0.0,0.0,26151.0,587.0,26738.0,07:25:38,318.0,0,NaN
1,112947202236269500,07:31:53,07:31:53,8742,2,0,0,NaN,BEEKKANT,NaN,50.853386,4.322974,NaN,NaN,0.0,1.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27141,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8742,BEEKKANT,BEEKKANT,POINT (146776.5 171444.3),POINT (146633.5 170956.4),508.424439,11-09-2021,07:32:21,0.0,8742,8742,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26308.0,833.0,27141.0,07:32:21,28.0,0,NaN
2,112947202236269500,07:33:33,07:33:33,8292,3,0,0,NaN,ETANGS NOIRS,NaN,50.857125,4.333143,NaN,NaN,0.0,12.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27233,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8292,ETANGS NOIRS,ZWARTE VIJVERS,POINT (147492.7 171859.9),POINT (146776.5 171444.3),828.049395,11-09-2021,07:33:53,0.0,8292,8292,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26370.0,863.0,27233.0,07:33:53,20.0,0,NaN
3,112947202236269500,07:34:44,07:34:44,8282,4,0,0,NaN,COMTE DE FLANDRE,NaN,50.854705,4.340542,NaN,NaN,0.0,13.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27295,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8282,COMTE DE FLANDRE,GRAAF VAN VLAAND.,POINT (148013.6 171590.4),POINT (147492.7 171859.9),586.487050,11-09-2021,07:34:55,0.0,8282,8282,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26432.0,863.0,27295.0,07:34:55,11.0,0,NaN
4,112947202236269500,07:36:08,07:36:08,8272,5,0,0,NaN,SAINTE-CATHERINE,NaN,50.851900,4.348012,NaN,NaN,0.0,14.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27385,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8272,SAINTE-CATHERINE,SINT-KATELIJNE,POINT (148539.5 171278.2),POINT (148013.6 171590.4),611.587811,11-09-2021,07:36:25,0.0,8272,8272,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26524.0,861.0,27385.0,07:36:25,17.0,0,NaN


## Getting rid of useless columns

#### Creating json file

In [ ]:
col = ["pickup_type", "drop_off_type", "stop_code", "stop_desc", "stop_lat", "stop_lon", "zone_id", "stop_url", "location_type", "parent_station","block_id","shape_id","route_desc", "route_url", "route_color", "route_text_color", "stop_id_clean_c", "stop_id__current","start_date", "end_date","label", "lag_-1", "lag_+1", "t_lag_before", "t_diff(sec)", "diff_time", "arrival_time", "route_type","monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday", "descr_fr","descr_nl","geometry","geom_position_before","stop_code__terminus",  "stop_desc__terminus", "stop_lat__terminus", "stop_lon__terminus","zone_id__terminus", "stop_url__terminus","location_type__terminus", "parent_station__terminus"]
col_to_drop_json = json.dumps(col)
with open("useless_col.json", "w") as outfile:
    outfile.write(col_to_drop_json)

In [3]:
uc = open("useless_col.json",'r')
col_to_drop = json.loads(uc.read())

### Removing the columns

In [4]:
sp_cleaned = sp.drop(
    columns = col_to_drop)
sp_cleaned.head()

,trip_id,departure_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,time_seconds,headway_min,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,adjusted_arrival_time(ts),adjusted_arrival_time,vehicule_id,approx_rec
0,112947202236269500,07:30:56,8733,1,GARE DE L'OUEST,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,26738,15.0,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,NaN,11-09-2021,07:25:38,0.0,8733,8161,STOCKEL,8161,saturday,1,M,26738.0,07:25:38,0,NaN
1,112947202236269500,07:31:53,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,27141,15.0,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,508.424439,11-09-2021,07:32:21,0.0,8742,8161,STOCKEL,8161,saturday,1,M,27141.0,07:32:21,0,NaN
2,112947202236269500,07:33:33,8292,3,ETANGS NOIRS,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,27233,15.0,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,828.049395,11-09-2021,07:33:53,0.0,8292,8161,STOCKEL,8161,saturday,1,M,27233.0,07:33:53,0,NaN
3,112947202236269500,07:34:44,8282,4,COMTE DE FLANDRE,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,27295,15.0,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,586.487050,11-09-2021,07:34:55,0.0,8282,8161,STOCKEL,8161,saturday,1,M,27295.0,07:34:55,0,NaN
4,112947202236269500,07:36:08,8272,5,SAINTE-CATHERINE,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,27385,15.0,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,611.587811,11-09-2021,07:36:25,0.0,8272,8161,STOCKEL,8161,saturday,1,M,27385.0,07:36:25,0,NaN


In [5]:
sp_cleaned.shape


(2121, 34)

## Computing Delays

#### Turning time into sec

In [5]:
def turn_hour_to_seconds(time_in_hour:str, timetable:pd.DataFrame, col_name:str ="time_seconds", format24:bool= True) -> pd.DataFrame:
    timetable = timetable.dropna(subset=time_in_hour)
    time = timetable[time_in_hour].str.split(":", expand=True)
    timetable[col_name] = (
        time[0].astype(int) * 3600 + time[1].astype(int) * 60 + time[2].astype(int)
    )
    if format24:
        timetable[col_name] = np.where(
            timetable[col_name] < 7200,
            timetable[col_name] + 86400,
            timetable[col_name] 
        )

    return timetable

In [6]:
sp_cleaned = turn_hour_to_seconds("departure_time", sp_cleaned, "time_seconds")
sp_cleaned = turn_hour_to_seconds("adjusted_arrival_time", sp_cleaned, "adjusted_arrival_time(ts)")

In [7]:
sp_cleaned= sp_cleaned.rename(columns= {"departure_time":"theoretical_time", "time_seconds": "th_time_sec", "adjusted_arrival_time": "adj_real_time", "adjusted_arrival_time(ts)": "real_time_sec"})

In [8]:
sp_cleaned.sort_values(by= "real_time_sec" )

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_min,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec
1617,112947241236269500,07:00:56,8733,1,GARE DE L'OUEST,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,25256,14.4,cluster_0,15.0,0,2021-09-04,2021-09-18,18-09-2021,NaN,18-09-2021,06:56:14,0.0,8733,8162,STOCKEL,8162,saturday,1,M,24974,06:56:14,77,NaN
1596,112947241236269500,07:00:56,8733,1,GARE DE L'OUEST,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,25256,14.4,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,NaN,11-09-2021,06:56:52,0.0,8733,8162,STOCKEL,8162,saturday,1,M,25012,06:56:52,76,NaN
1597,112947241236269500,07:01:53,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,25313,14.4,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,508.424439,11-09-2021,07:01:58,0.0,8742,8162,STOCKEL,8162,saturday,1,M,25318,07:01:58,76,NaN
1618,112947241236269500,07:01:53,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,25313,14.4,cluster_0,15.0,0,2021-09-04,2021-09-18,18-09-2021,508.424439,18-09-2021,07:02:01,0.0,8742,8162,STOCKEL,8162,saturday,1,M,25321,07:02:01,77,NaN
1619,112947241236269500,07:03:33,8292,3,ETANGS NOIRS,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,25413,14.4,cluster_0,15.0,0,2021-09-04,2021-09-18,18-09-2021,828.049395,18-09-2021,07:03:35,0.0,8292,8162,STOCKEL,8162,saturday,1,M,25415,07:03:35,77,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592,112947240236269500,24:32:05,8132,18,VANDERVELDE,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,88325,20.0,cluster_4,17.0,0,2021-09-04,2021-09-18,18-09-2021,788.888256,19-09-2021,00:32:57,0.0,8132,8162,STOCKEL,8162,sunday,1,M,88377,00:32:57,75,NaN
1572,112947240236269500,24:33:11,8142,19,ALMA,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,88391,20.0,cluster_4,17.0,0,2021-09-04,2021-09-18,11-09-2021,514.791385,12-09-2021,00:33:53,0.0,8142,8161,STOCKEL,8161,sunday,1,M,88433,00:33:53,74,NaN
1593,112947240236269500,24:33:11,8142,19,ALMA,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,88391,20.0,cluster_4,17.0,0,2021-09-04,2021-09-18,18-09-2021,514.791385,19-09-2021,00:33:58,0.0,8142,8162,STOCKEL,8162,sunday,1,M,88438,00:33:58,75,NaN
1594,112947240236269500,24:34:21,8152,20,CRAINHEM,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,88461,20.0,cluster_4,17.0,0,2021-09-04,2021-09-18,18-09-2021,424.146979,19-09-2021,00:35:34,0.0,8152,8162,STOCKEL,8162,sunday,1,M,88534,00:35:34,75,NaN


In [9]:
sp_cleaned["delay"] = sp_cleaned["real_time_sec"] - sp_cleaned["th_time_sec"]
sp_cleaned.head()

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_min,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,delay
0,112947202236269500,07:30:56,8733,1,GARE DE L'OUEST,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,27056,15.0,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,NaN,11-09-2021,07:25:38,0.0,8733,8161,STOCKEL,8161,saturday,1,M,26738,07:25:38,0,NaN,-318
1,112947202236269500,07:31:53,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,27113,15.0,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,508.424439,11-09-2021,07:32:21,0.0,8742,8161,STOCKEL,8161,saturday,1,M,27141,07:32:21,0,NaN,28
2,112947202236269500,07:33:33,8292,3,ETANGS NOIRS,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,27213,15.0,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,828.049395,11-09-2021,07:33:53,0.0,8292,8161,STOCKEL,8161,saturday,1,M,27233,07:33:53,0,NaN,20
3,112947202236269500,07:34:44,8282,4,COMTE DE FLANDRE,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,27284,15.0,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,586.487050,11-09-2021,07:34:55,0.0,8282,8161,STOCKEL,8161,saturday,1,M,27295,07:34:55,0,NaN,11
4,112947202236269500,07:36:08,8272,5,SAINTE-CATHERINE,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,27368,15.0,cluster_0,15.0,0,2021-09-04,2021-09-18,11-09-2021,611.587811,11-09-2021,07:36:25,0.0,8272,8161,STOCKEL,8161,saturday,1,M,27385,07:36:25,0,NaN,17


## Computing EWT

EWT = AWT - SWT
SWT = somme s_headways² / 2* somme s_headways
AWT = somme r_hadways²/ 2* somme r_headways

### Compute real headways

In [10]:
def compute_headway(
    df: pd.DataFrame,
    headway_col_name: str = "headway_min",
    time_sec_col_name: str = "time_seconds",
) -> pd.DataFrame:
    df[headway_col_name] = (
        df[time_sec_col_name].sort_values()
        - df[time_sec_col_name].sort_values().shift()
    ) / 60
    return df

sample_headway = []
sg = sp_cleaned.groupby(by= ["stop_id", "route_id", "direction_id", "date_normalized"]).apply(compute_headway)
for name,group in tqdm(sg):
    sample_headway.append(compute_headway(group.sort_values("th_time_sec"), "headway_real", "real_time_sec"))

del sg

sample_headway = pd.concat(sample_headway).reset_index(drop=True)

sample_headway.head()


100%|██████████| 83/83 [00:01<00:00, 55.59it/s]


,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_min,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,delay,headway_real
0,112947218236269500,09:23:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,33831,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,09:24:35,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,33875,09:24:35,30,NaN,44,NaN
1,112947228236269500,12:14:41,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,44081,7.500000,cluster_2,7.5,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,12:14:01,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,44517,12:21:57,50,1.0,436,177.366667
2,112947235236269500,18:59:24,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,68364,7.316667,cluster_2,7.5,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,18:59:29,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,68369,18:59:29,64,NaN,5,397.533333
3,112947218236269500,09:23:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,33831,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,18-09-2021,588.817544,18-09-2021,09:25:42,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,33942,09:25:42,31,NaN,111,NaN
4,112947228236269500,12:14:41,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,44081,7.500000,cluster_2,7.5,1,2021-09-04,2021-09-18,18-09-2021,588.817544,18-09-2021,12:14:18,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,44058,12:14:18,51,NaN,-23,168.600000


In [20]:
test = sample_headway[(sample_headway.stop_id_cleaned == "8152") & (sample_headway.direction_id == 0) & (sample_headway.date_normalized == "11-09-2021")]
fig = px.scatter(
    test.sort_values("th_time_sec"),
    x= "adj_real_time",
    y= "headway_real",
    color= "date_normalized"
)
fig.show()

#### Square roots headways

In [39]:
sample_headway["s_headway_square"] = sample_headway["headway_min"]**2
sample_headway["r_headway_square"] = sample_headway["headway_real"]**2
sample_headway.head()

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_min,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,delay,headway_real,s_headway_square,r_headway_square
0,112947218236269500,09:23:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,33831,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,09:24:35,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,33875,09:24:35,30,NaN,44,NaN,100.000000,NaN
1,112947228236269500,12:14:41,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,44081,7.500000,cluster_2,7.5,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,12:14:01,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,44517,12:21:57,50,1.0,436,177.366667,56.250000,31458.934444
2,112947235236269500,18:59:24,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,68364,7.316667,cluster_2,7.5,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,18:59:29,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,68369,18:59:29,64,NaN,5,397.533333,53.533611,158032.751111
3,112947218236269500,09:23:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,33831,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,18-09-2021,588.817544,18-09-2021,09:25:42,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,33942,09:25:42,31,NaN,111,NaN,100.000000,NaN
4,112947228236269500,12:14:41,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,44081,7.500000,cluster_2,7.5,1,2021-09-04,2021-09-18,18-09-2021,588.817544,18-09-2021,12:14:18,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,44058,12:14:18,51,NaN,-23,168.600000,56.250000,28425.960000


In [40]:
SWT = sample_headway.groupby(by= ["stop_id", "route_id", "direction_id", "date_normalized", "clusters"]).apply(lambda x: x["s_headway_square"].sum()/(2*(x["headway_min"].sum())))
AWT = sample_headway.groupby(by= ["stop_id", "route_id", "direction_id", "date_normalized", "clusters"]).apply(lambda x: x["r_headway_square"].sum()/(2*(x["headway_real"].sum())))

SWT.name = "SWT"
AWT.name = "AWT"

sample_headway = sample_headway.merge(
    SWT,
    on= ["stop_id", "route_id", "direction_id", "date_normalized", "clusters"],
    how= 'left'
).merge(
    AWT,
    on= ["stop_id", "route_id", "direction_id", "date_normalized", "clusters"],
    how= 'left'
)

# for name,group in clusters:
#     print(group.columns)
#     group["SWT"] = group.apply(lambda x: x["s_headway_square"].sum()/(2*(x["headway_min"].sum())))
#     group["AWT"] = group.apply(lambda x: x["r_headway_square"].sum()/(2*(x["headway_real"].sum())))



C:\Users\bagui\AppData\Local\Temp\ipykernel_5040\3714213512.py:2: RuntimeWarning:

invalid value encountered in double_scalars



In [42]:
sample_headway["EWT"]= sample_headway["AWT"] - sample_headway["SWT"]
sample_headway

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_min,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,delay,headway_real,s_headway_square,r_headway_square,SWT,AWT,EWT
0,112947218236269500,09:23:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,33831,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,09:24:35,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,33875,09:24:35,30,NaN,44,NaN,100.000000,NaN,5.000000,NaN,NaN
1,112947228236269500,12:14:41,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,44081,7.500000,cluster_2,7.5,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,12:14:01,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,44517,12:21:57,50,1.0,436,177.366667,56.250000,31458.934444,3.704734,164.804040,161.099307
2,112947235236269500,18:59:24,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,68364,7.316667,cluster_2,7.5,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,18:59:29,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,68369,18:59:29,64,NaN,5,397.533333,53.533611,158032.751111,3.704734,164.804040,161.099307
3,112947218236269500,09:23:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,33831,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,18-09-2021,588.817544,18-09-2021,09:25:42,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,33942,09:25:42,31,NaN,111,NaN,100.000000,NaN,5.000000,NaN,NaN
4,112947228236269500,12:14:41,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,44081,7.500000,cluster_2,7.5,1,2021-09-04,2021-09-18,18-09-2021,588.817544,18-09-2021,12:14:18,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,44058,12:14:18,51,NaN,-23,168.600000,56.250000,28425.960000,3.704734,168.709194,165.004461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982,112947238236269500,21:41:36,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,78096,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,508.424439,18-09-2021,21:41:26,0.0,8742,8161,STOCKEL,8161,saturday,1,M,78404,21:46:44,71,1.0,308,29.816667,100.000000,889.033611,6.329689,15.532295,9.202606
1983,112947213236269500,22:21:36,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,80496,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,508.424439,18-09-2021,22:22:59,0.0,8742,8161,STOCKEL,8161,saturday,1,M,80739,22:25:39,23,1.0,243,38.916667,100.000000,1514.506944,6.329689,15.532295,9.202606
1984,112947239236269500,22:51:36,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,82296,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,508.424439,18-09-2021,22:51:45,0.0,8742,8161,STOCKEL,8161,saturday,1,M,82652,22:57:32,73,1.0,356,31.883333,100.000000,1016.546944,6.329689,15.532295,9.202606
1985,112947214236269500,23:31:36,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,84696,17.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,508.424439,18-09-2021,23:31:40,0.0,8742,8161,STOCKEL,8161,saturday,1,M,84700,23:31:40,25,NaN,4,34.133333,289.000000,1165.084444,6.329689,15.532295,9.202606


### Get EWT

In [16]:
def compute_EWT(
    timetable: pd.DataFrame,
    stop_col: str = "stop_id",
    route_col: str = "route_id",
    direction_col: str = "direction_id",
    date_col: str = "date_normalized",
    cluster_col: str = "clusters",
    s_headway_col: str = "headway_min",
    a_headway_col:str = "headway_real") -> pd.DataFrame:

    timetable["s_headway_square"] = timetable[s_headway_col]**2
    timetable["a_headway_square"] = timetable[a_headway_col]**2

    SWT = timetable.groupby(by= [stop_col, route_col, direction_col, date_col, cluster_col]).apply(lambda x: x["s_headway_square"].sum()/(2*(x[s_headway_col].sum())))
    AWT = timetable.groupby(by= [stop_col, route_col, direction_col, date_col, cluster_col]).apply(lambda x: x["a_headway_square"].sum()/(2*(x[a_headway_col].sum())))

    SWT.name = "SWT"
    AWT.name = "AWT"

    timetable = timetable.merge(
        SWT,
        on= [stop_col, route_col, direction_col, date_col, cluster_col],
        how= 'left'
    ).merge(
        AWT,
        on= [stop_col, route_col, direction_col, date_col, cluster_col],
        how= 'left'
    )

    timetable["EWT"]= timetable["AWT"] - timetable["SWT"]

    timetable = timetable.drop(columns=["s_headway_square","a_headway_square"])

    return timetable

In [17]:
final_sample = compute_EWT(timetable = sample_headway)
final_sample

C:\Users\bagui\AppData\Local\Temp\ipykernel_23840\4294571667.py:15: RuntimeWarning: invalid value encountered in double_scalars
  AWT = timetable.groupby(by= [stop_col, route_col, direction_col, date_col, cluster_col]).apply(lambda x: x["a_headway_square"].sum()/(2*(x[a_headway_col].sum())))


,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_min,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,delay,headway_real,SWT,AWT,EWT
0,112947218236269500,09:23:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,33831,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,09:24:35,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,33875,09:24:35,30,NaN,44,NaN,5.000000,NaN,NaN
1,112947228236269500,12:14:41,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,44081,7.500000,cluster_2,7.5,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,12:14:01,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,44517,12:21:57,50,1.0,436,177.366667,3.704734,164.804040,161.099307
2,112947235236269500,18:59:24,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,68364,7.316667,cluster_2,7.5,1,2021-09-04,2021-09-18,11-09-2021,588.817544,11-09-2021,18:59:29,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,68369,18:59:29,64,NaN,5,397.533333,3.704734,164.804040,161.099307
3,112947218236269500,09:23:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,33831,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,18-09-2021,588.817544,18-09-2021,09:25:42,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,33942,09:25:42,31,NaN,111,NaN,5.000000,NaN,NaN
4,112947228236269500,12:14:41,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,44081,7.500000,cluster_2,7.5,1,2021-09-04,2021-09-18,18-09-2021,588.817544,18-09-2021,12:14:18,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,44058,12:14:18,51,NaN,-23,168.600000,3.704734,168.709194,165.004461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982,112947238236269500,21:41:36,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,78096,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,508.424439,18-09-2021,21:41:26,0.0,8742,8161,STOCKEL,8161,saturday,1,M,78404,21:46:44,71,1.0,308,29.816667,6.329689,15.532295,9.202606
1983,112947213236269500,22:21:36,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,80496,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,508.424439,18-09-2021,22:22:59,0.0,8742,8161,STOCKEL,8161,saturday,1,M,80739,22:25:39,23,1.0,243,38.916667,6.329689,15.532295,9.202606
1984,112947239236269500,22:51:36,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,82296,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,508.424439,18-09-2021,22:51:45,0.0,8742,8161,STOCKEL,8161,saturday,1,M,82652,22:57:32,73,1.0,356,31.883333,6.329689,15.532295,9.202606
1985,112947214236269500,23:31:36,8742,2,BEEKKANT,2,236269500,STOCKEL,0,1,GARE DE L'OUEST - STOCKEL,84696,17.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,508.424439,18-09-2021,23:31:40,0.0,8742,8161,STOCKEL,8161,saturday,1,M,84700,23:31:40,25,NaN,4,34.133333,6.329689,15.532295,9.202606


## Insights

In [2]:
import pandas as pd
import numpy as np



In [36]:
df = pd.read_pickle("C:/Users/bagui/OneDrive - Université Libre de Bruxelles/MS-BGDA/Data mining/STIB Projet/STIB-Network-QoS-Analysis/data/computed/matches_EWT_delay_subsampled.pkl")

In [37]:
pd.options.display.max_columns = None
df.head()


,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,delay,delay_label
3755153,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.000000,cluster_4,10.00,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat
3755159,112949813236270000,05:09:07,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,18547,NaN,cluster_0,4.75,1,2021-09-01,2021-09-17,07-09-2021,588.817544,07-09-2021,05:07:56,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,18476,05:07:56,111546,NaN,NaN,2.796185,8.242382,5.446197,-71,adv
3755163,112950843236270000,06:07:09,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,22029,2.483333,cluster_0,4.75,1,2021-09-01,2021-09-17,07-09-2021,588.817544,07-09-2021,06:07:20,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,22304,06:11:44,113846,1.0,0.733333,2.796185,8.242382,5.446197,275,lat
3755164,112950421236270000,06:12:25,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,22345,5.266667,cluster_0,4.75,1,2021-09-01,2021-09-17,07-09-2021,588.817544,07-09-2021,06:11:00,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,22260,06:11:00,113386,NaN,17.200000,2.796185,8.242382,5.446197,-85,adv
3755336,112952547236270000,05:56:26,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,21386,3.183333,cluster_0,4.75,1,2021-09-01,2021-09-17,08-09-2021,588.817544,08-09-2021,05:55:10,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,21310,05:55:10,113907,NaN,10.933333,2.531103,4.595001,2.063899,-76,adv


In [38]:
df["EWT_label"] = np.where(df["EWT"] >= 1, "irregular", "regular")
df

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,delay,delay_label,EWT_label
3755153,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.000000,cluster_4,10.00,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular
3755159,112949813236270000,05:09:07,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,18547,NaN,cluster_0,4.75,1,2021-09-01,2021-09-17,07-09-2021,588.817544,07-09-2021,05:07:56,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,18476,05:07:56,111546,NaN,NaN,2.796185,8.242382,5.446197,-71,adv,irregular
3755163,112950843236270000,06:07:09,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,22029,2.483333,cluster_0,4.75,1,2021-09-01,2021-09-17,07-09-2021,588.817544,07-09-2021,06:07:20,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,22304,06:11:44,113846,1.0,0.733333,2.796185,8.242382,5.446197,275,lat,irregular
3755164,112950421236270000,06:12:25,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,22345,5.266667,cluster_0,4.75,1,2021-09-01,2021-09-17,07-09-2021,588.817544,07-09-2021,06:11:00,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,22260,06:11:00,113386,NaN,17.200000,2.796185,8.242382,5.446197,-85,adv,irregular
3755336,112952547236270000,05:56:26,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,21386,3.183333,cluster_0,4.75,1,2021-09-01,2021-09-17,08-09-2021,588.817544,08-09-2021,05:55:10,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,21310,05:55:10,113907,NaN,10.933333,2.531103,4.595001,2.063899,-76,adv,irregular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3773839,112949518236269500,23:21:38,8031,14,PARC,2,236269500,ERASME,1,1,GARE DE L'OUEST - STOCKEL,84098,10.000000,cluster_3,10.00,1,2021-09-04,2021-09-18,18-09-2021,471.506564,18-09-2021,23:25:27,0.0,8031,8642,ERASME,8642,saturday,1,M,84327,23:25:27,497,NaN,11.166667,5.745127,5.820706,0.075579,229,lat,regular
3773840,112947678236269500,23:31:38,8031,14,PARC,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,84698,10.000000,cluster_3,10.00,1,2021-09-04,2021-09-18,18-09-2021,471.506564,18-09-2021,23:31:40,0.0,8031,8731,GARE DE L'OUEST,8731,saturday,1,M,84700,23:31:40,307,NaN,6.216667,5.745127,5.820706,0.075579,2,nan,regular
3773841,112947668236269500,23:46:38,8031,14,PARC,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,85598,15.000000,cluster_3,10.00,1,2021-09-04,2021-09-18,18-09-2021,471.506564,18-09-2021,23:48:46,0.0,8031,8731,GARE DE L'OUEST,8731,saturday,1,M,85726,23:48:46,287,NaN,17.100000,5.745127,5.820706,0.075579,128,lat,regular
3773842,112947687236269500,24:06:38,8031,14,PARC,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,86798,20.000000,cluster_3,10.00,1,2021-09-04,2021-09-18,18-09-2021,471.506564,19-09-2021,00:06:36,0.0,8031,8731,GARE DE L'OUEST,8731,sunday,1,M,86796,00:06:36,325,NaN,17.833333,5.745127,5.820706,0.075579,-2,adv,regular


### Global and for regularity clusters

In [19]:
# On time positionning (idealemnent, rajouter un threshold with thhresh hold)

OTP = round((df[df.regularity == 0].delay_label.value_counts()/df[df.regularity == 0].shape[0])*100, 2).to_dict()
OTP

{'lat': 55.11, 'nan': 25.57, 'adv': 19.32}

In [22]:
EWT = round((df[df.regularity == 1].EWT_label.value_counts()/df[df.regularity == 1].shape[0])*100, 2).to_dict()

In [23]:
EWT 

{'regular': 72.7, 'irregular': 27.3}

In [24]:
EWT_global = round((df.EWT_label.value_counts()/df.shape[0])*100, 2).to_dict()
EWT_global

{'regular': 71.58, 'irregular': 28.42}

### /Line

In [46]:
EWT_line = df[df.regularity == 1].groupby(by = ["route_short_name"]).apply(lambda x: x["EWT"].median())
EWT_line.name = 'EWT_line'

df = df.merge(
    EWT_line,
    'left',
    'route_short_name'
)

In [48]:
df["EWT_line_label"] = np.where(df["EWT_line"] >= 1, "irregular", "regular")

In [50]:
line_prob= round((df[df.regularity == 1].EWT_line_label.value_counts()/df[df.regularity == 1].shape[0])*100, 2).to_dict()

In [51]:
line_prob

{'regular': 100.0}

In [63]:
best_line = df[(df.regularity == 1) & (df.EWT_line_label == "regular")].sort_values(by= 'EWT_line', ascending = True).iloc[0].route_short_name

In [64]:
worst_line = df[(df.regularity == 1) & (df.EWT_line_label == "irregular")].sort_values(by= 'EWT_line', ascending = False).iloc[0].route_short_name

'1'

### /stop

In [69]:
EWT_stop = df[df.regularity == 1].groupby(by = ["stop_id", "route_short_name"]).apply(lambda x: x["EWT"].median())
EWT_stop.name = 'EWT_stop'

df = df.merge(
    EWT_stop,
    'left',
    'route_short_name'
)

df

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,delay,delay_label,EWT_label,EWT_line,EWT_line_label,EWT_stop
0,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.0,cluster_4,10.0,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular,0.654696,regular,0.496864
1,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.0,cluster_4,10.0,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular,0.654696,regular,0.595723
2,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.0,cluster_4,10.0,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular,0.654696,regular,0.664213
3,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.0,cluster_4,10.0,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular,0.654696,regular,0.736568
4,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.0,cluster_4,10.0,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular,0.654696,regular,0.733240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46495,112947722236269500,24:26:38,8031,14,PARC,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,87998,20.0,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,471.506564,19-09-2021,00:27:11,0.0,8031,8731,GARE DE L'OUEST,8731,sunday,1,M,88031,00:27:11,395,NaN,20.583333,5.745127,5.820706,0.075579,33,adv,regular,0.654696,regular,0.496864
46496,112947722236269500,24:26:38,8031,14,PARC,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,87998,20.0,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,471.506564,19-09-2021,00:27:11,0.0,8031,8731,GARE DE L'OUEST,8731,sunday,1,M,88031,00:27:11,395,NaN,20.583333,5.745127,5.820706,0.075579,33,adv,regular,0.654696,regular,0.595723
46497,112947722236269500,24:26:38,8031,14,PARC,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,87998,20.0,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,471.506564,19-09-2021,00:27:11,0.0,8031,8731,GARE DE L'OUEST,8731,sunday,1,M,88031,00:27:11,395,NaN,20.583333,5.745127,5.820706,0.075579,33,adv,regular,0.654696,regular,0.664213
46498,112947722236269500,24:26:38,8031,14,PARC,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,87998,20.0,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,471.506564,19-09-2021,00:27:11,0.0,8031,8731,GARE DE L'OUEST,8731,sunday,1,M,88031,00:27:11,395,NaN,20.583333,5.745127,5.820706,0.075579,33,adv,regular,0.654696,regular,0.736568


In [70]:
df[(df.stop_id == "8011") & (df.route_short_name == "1")]

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,date,time,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,delay,delay_label,EWT_label,EWT_line,EWT_line_label,EWT_stop
0,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.0,cluster_4,10.0,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular,0.654696,regular,0.496864
1,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.0,cluster_4,10.0,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular,0.654696,regular,0.595723
2,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.0,cluster_4,10.0,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular,0.654696,regular,0.664213
3,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.0,cluster_4,10.0,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular,0.654696,regular,0.736568
4,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.0,cluster_4,10.0,1,2021-09-01,2021-09-17,06-09-2021,588.817544,06-09-2021,23:13:06,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat,irregular,0.654696,regular,0.733240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9480,112947722236269500,24:28:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,88133,20.0,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,588.817544,19-09-2021,00:28:46,0.0,8011,8731,GARE DE L'OUEST,8731,sunday,1,M,88126,00:28:46,395,NaN,19.533333,5.656681,6.730830,1.074149,-7,adv,irregular,0.654696,regular,0.496864
9481,112947722236269500,24:28:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,88133,20.0,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,588.817544,19-09-2021,00:28:46,0.0,8011,8731,GARE DE L'OUEST,8731,sunday,1,M,88126,00:28:46,395,NaN,19.533333,5.656681,6.730830,1.074149,-7,adv,irregular,0.654696,regular,0.595723
9482,112947722236269500,24:28:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,88133,20.0,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,588.817544,19-09-2021,00:28:46,0.0,8011,8731,GARE DE L'OUEST,8731,sunday,1,M,88126,00:28:46,395,NaN,19.533333,5.656681,6.730830,1.074149,-7,adv,irregular,0.654696,regular,0.664213
9483,112947722236269500,24:28:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,88133,20.0,cluster_3,10.0,1,2021-09-04,2021-09-18,18-09-2021,588.817544,19-09-2021,00:28:46,0.0,8011,8731,GARE DE L'OUEST,8731,sunday,1,M,88126,00:28:46,395,NaN,19.533333,5.656681,6.730830,1.074149,-7,adv,irregular,0.654696,regular,0.736568


In [67]:
worse_stop = df[(df.regularity == 1) & (df.EWT_label == "irregular")].sort_values(by= 'EWT', ascending = False).iloc[0].stop_name

In [68]:
worse_stop

'DE BROUCKERE'

In [79]:
data = pd.read_pickle("C:/Users/bagui/OneDrive - Université Libre de Bruxelles/MS-BGDA/Data mining/STIB Projet/STIB-Network-QoS-Analysis/data/computed/observed_schedule_dist_0.pkl")

In [81]:
data

,date,time,route_short_name,distance_from_point,stop_id,stop_id_terminus,route_type,current_stop_name,terminus_stop_name,time_seconds,lag_-1,adjusted_arrival_time(ts),adjusted_arrival_time,headway_min
0,06-09-2021,09:56:18,13,48,1008,4111,3,RENIERS,ETANGS NOIRS,35778,NaN,35766.923,09:56:06,NaN
1,07-09-2021,00:03:04,13,48,1008,4111,3,RENIERS,ETANGS NOIRS,184,NaN,172.923,00:02:52,NaN
2,07-09-2021,12:14:33,13,0,1008,4111,3,RENIERS,ETANGS NOIRS,44073,48.0,44073.000,12:14:33,731.667950
3,07-09-2021,14:13:21,13,48,1008,4111,3,RENIERS,ETANGS NOIRS,51201,78.0,51189.923,14:13:09,118.615383
4,07-09-2021,14:24:36,13,46,1008,4111,3,RENIERS,ETANGS NOIRS,51876,48.0,51865.385,14:24:25,11.257700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3446289,12-09-2021,16:37:27,80,0,9983,2893,3,MEUDON,TOMBERG,59847,70.0,59847.000,16:37:27,20.966667
3446290,12-09-2021,16:53:47,80,0,9983,2893,3,MEUDON,TOMBERG,60827,158.0,60827.000,16:53:47,16.333333
3446291,12-09-2021,17:17:37,80,0,9983,2893,3,MEUDON,TOMBERG,62257,196.0,62257.000,17:17:37,23.833333
3446292,12-09-2021,17:36:04,80,0,9983,2893,3,MEUDON,TOMBERG,63364,107.0,63364.000,17:36:04,18.450000


In [80]:
data[(data.stop_id == "8011") & (data.route_short_name == "1")].route_long_name.value_counts()

AttributeError: 'DataFrame' object has no attribute 'route_long_name'